## Illustris web API details and example scripts available at: http://www.illustris-project.org/data/docs/api/

In [ ]:
!pip install astropy

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import h5py
import astropy.table as atpy

import requests
import os

%matplotlib inline

In [ ]:
#input your own api key; your key is listed here after login: http://www.illustris-project.org/data/
apikey=

In [ ]:
def get(path, params=None):
 
    # make HTTP GET request to path
    headers = {"api-key":str(apikey)}
    r = requests.get(path, params=params, headers=headers)

    # raise exception if response code is not HTTP SUCCESS (200)
    r.raise_for_status()

    if r.headers['content-type'] == 'application/json':
        return r.json() # parse json responses automatically

    if 'content-disposition' in r.headers:
        filename = r.headers['content-disposition'].split("filename=")[1]
        with open(filename, 'wb') as f:
            f.write(r.content)
        return filename # return the filename string

    return r

In [ ]:
#simulation
Isim=1

#snapshot number, snapNum=135 -> z=0 
snapNum = 135

#cosmology
H0=70.
h=H0/100.

# 1. Meta data: how many groups and subhalos exist at a given snapshot?


In [ ]:
#url to snapshot detail
url='http://www.illustris-project.org/api/Illustris-'+str(Isim)+'/snapshots/'+str(snapNum)
print(url)

In [ ]:
#access this info from web
metadata=get(url) 

#available headers
print(metadata.keys())


In [ ]:
print('redshift at snapshot='+str(snapNum)+':',metadata['redshift'])

print('number of subhalos at snapshot='+str(snapNum)+':', metadata['num_groups_subfind'])
print('number of FOF groups at snapshot='+str(snapNum)+':', metadata['num_groups_fof'])

# 2. Halo catalogs: information about a specific FOF group
FOF halos data specifications: http://www.illustris-project.org/data/docs/specifications/#sec2a

In [ ]:
#ID of FOF group we'll use for the example
groupID=1000

#weblink to group info
url='http://www.illustris-project.org/api/Illustris-'+str(Isim)+'/snapshots/'+str(snapNum)+'/halos/'+str(groupID)+'/info.json'
print(url)


In [ ]:
#access this info
group=get(url)['Group']
print(group.keys())

In [ ]:
#M200 mass
print('group M200:',group['Group_M_Crit200'])

### **Interactive break**: 
print out other properties of this group 

In [ ]:
#print out other properties of this group here
#

In [ ]:
#subhaloID of central subhalo of FOF group
bcgID=group['GroupFirstSub']
print('ID of central galaxy:',bcgID)

In [ ]:
#subhaloIDs of all subhalos of group
Nsubs=group['GroupNsubs']
subhaloIDlist=np.arange(bcgID,bcgID+Nsubs)
print('number of subhalos in group:',Nsubs)
print('subhalo ID list:',subhaloIDlist)

In [ ]:
#to check:
url='http://www.illustris-project.org/api/Illustris-'+str(Isim)+'/snapshots/135/halos/'+str(groupID)+'/'
print(url)

# 3. Subhalo catalog: information about a specific subhalo
Subhalos data specifications: http://www.illustris-project.org/data/docs/specifications/#sec2b

In [ ]:
#let's access the information for the central subhalo of our group
subhaloID=bcgID
print(subhaloID)

#weblink to subhalo info
url='http://www.illustris-project.org/api/Illustris-'+str(Isim)+'/snapshots/'+str(snapNum)+'/subhalos/'+str(subhaloID)+'/info.json'
print(url)

subhalo=get(url)['Subhalo']
print(subhalo.keys())
    

In [ ]:
#Total mass of subhalo
print('subhalo mass:',subhalo['SubhaloMass'])


### **Interactive break**: 
print out other properties of this subhalo 

In [ ]:
#print out other properties of this subhalo here
#

# 4. Merger trees
Merger trees data specifications: http://www.illustris-project.org/data/docs/specifications/#sec3a

In [ ]:
#let's look at the MPB merger tree for our central subhalo
subhaloID=bcgID


#MPB merger tree for bcg -- NB: MPB specified in url
url='http://www.illustris-project.org/api/Illustris-'+str(Isim)+'/snapshots/'+str(snapNum)+'/subhalos/'+str(subhaloID)+'/sublink/mpb.hdf5'
mpb_filename=get(url)

#put tree data into another data structure: astropy tables
tree=atpy.Table()
with h5py.File(mpb_filename) as ft:
  for key in ft.keys():
    tree.add_column(atpy.Column(name=str(key), data=np.array(ft[str(key)])))

#remove tree file - hdf5 files remain in working directory otherwise
if os.path.isfile('./sublink_mpb_'+str(subhaloID)+'.hdf5')==True:
  os.remove('./sublink_mpb_'+str(subhaloID)+'.hdf5')


print(tree.columns)


In [ ]:
#print out some columns
print(tree['SnapNum','SubfindID','SubhaloGrNr','SubhaloMass'])

In [ ]:
#plot mass evolution of subhalo

fig1=plt.figure(1,(5,5))
fig1.clf()

ax=fig1.add_subplot(1,1,1)

plt.plot(tree['SnapNum'],tree['SubhaloMass'],'b')
plt.gca().invert_xaxis() 

plt.xlabel('SnapNum')
plt.ylabel('SubhaloMass')


### Q:
How can the above axis labels be improved?

### **Interactive break**: 
plot the evolution of other properties from the merger tree \

In [ ]:
#plot other tree properties here
#